In [ ]:
import json
import os
import glob
import pandas as pd
import statsmodels.api as sm
from statistics import mean
from collections import defaultdict
from patsy import dmatrices

DATA = "DC"
RECOMPUTE_DATA = False

In [ ]:
if RECOMPUTE_DATA:
    df_original = pd.read_csv(f"../data/{DATA}/all.txt.averaged_rt.annotation")
    df_original = df_original.sort_values(["article", "sent_id", "tokenN_in_sent"])
    df = df_original.copy()
    df = df[df["tokenN_in_sent"] > 0]
    df = df[df["is_first"] == False]

    # load results
    target_files = glob.glob(f"../results/tuned-lens/{DATA}/**/surprisal*.time", recursive=True)
    layer2score = {}
    model2layer2score = defaultdict(dict)
    for file in target_files:
        layer = file.split(".")[-2]
        model = file.split("/")[-2]

        if layer == "layerAverage":
            continue
        with open(file) as f:
            layer_avg_loglik = f.readlines()[1].strip().split()[-1]
            layer_avg_loglik = float(layer_avg_loglik)
            layer2score[layer] = layer_avg_loglik
            model2layer2score[model][layer] = layer_avg_loglik

    # load surprisals

    model2article2interest = defaultdict(dict)
    model2last_layer = {}
    model2best_layer = {}

    target_files = glob.glob(f"../results/tuned-lens/{DATA}/**/surprisal.json", recursive=True)
    for target_file in target_files:
        article2interest = json.load(open(target_file))
        model = target_file.split("/")[-2]
        model2article2interest[model] = article2interest
        layer2score = model2layer2score[model]
        print(model)
        model2last_layer[model] = sorted(layer2score.items(), key=lambda x: int(x[0].replace("layer", "").replace("Average", "-1")))[-1][0].replace("layer", "")
        model2best_layer[model] = sorted(layer2score.items(), key=lambda x: x[1])[-1][0].replace("layer", "")

    # compute residual errors

    def modeling(layer_id, article2interest):
        # print(layer_id)
        all_interests = [sup for _, sents_sups in sorted(article2interest[layer_id].items(), key=lambda x: int(x[0])) for sent_sups in sents_sups for sup in sent_sups]
        mean_surprisal = mean(all_interests)
        df = df_original.copy()

        df["interest"] = df_original.apply(lambda x: article2interest[layer_id][str(x["article"])][x["sent_id"]][x["tokenN_in_sent"]], axis=1)
        df["interest_prev_1"] = [mean_surprisal] + list(df["interest"])[:-1]
        df["interest_prev_1"] = df.apply(lambda x: x["interest_prev_1"] if x["tokenN_in_sent"] > 0 else mean_surprisal, axis=1)
        df["interest_prev_2"] = [mean_surprisal, mean_surprisal] + list(df["interest"])[:-2]
        df["interest_prev_2"] = df.apply(lambda x: x["interest_prev_2"] if x["tokenN_in_sent"] > 1 else mean_surprisal, axis=1)


        target_df = df.copy()
        target = "time"
        formula = f'{target} ~ interest + interest_prev_1 + interest_prev_2 + length + log_gmean_freq +  + length_prev_1 + log_gmean_freq_prev_1 + length_prev_2 + log_gmean_freq_prev_2'
        baseline_formula = f'{target} ~ interest_prev_1 + interest_prev_2 + length + log_gmean_freq  + length_prev_1 + log_gmean_freq_prev_1 + length_prev_2 + log_gmean_freq_prev_2'

        target_df = target_df[target_df["tokenN_in_sent"] > 0]
        target_df = target_df[target_df["is_first"] == False]

        y, X = dmatrices(formula, data=target_df, return_type='dataframe')
        mod = sm.OLS(y, X)
        res = mod.fit()

        y_baseline, X_baseline = dmatrices(baseline_formula, data=target_df, return_type='dataframe')
        mod_baseline = sm.OLS(y_baseline, X_baseline)
        res_baseline = mod_baseline.fit() 
        return  abs(res_baseline.resid) - abs(res.resid), res_baseline.mse_model - res.mse_model # higher is better


    model2layer2residuals = defaultdict(dict)
    model2layer2mse = defaultdict(dict)

    for model in model2last_layer.keys():
        print(model)
        last_layer = model2last_layer[model]
        article2interest = model2article2interest[model]
        last_residuals, mse = modeling(last_layer, article2interest)
        layer2mse = {}
        layer2residuals = {}
        for layer_id in article2interest.keys():
            residuals, mse = modeling(layer_id, article2interest)
            model2layer2residuals[model][layer_id] =  residuals - last_residuals
            model2layer2mse[model][layer_id] = mse

    
    dfs = []
    for model, layer2residuals in model2layer2residuals.items():
        df_regression = df.copy()
        print(model)
        best_residuals = layer2residuals[model2best_layer[model]]
        df_regression["residuals_best"] = best_residuals
        df_regression["model"] = model
        dfs.append(df_regression)
    df_regression = pd.concat(dfs)
    df_regression.to_csv(f"../results/{DATA}_error_analysis.csv", index=False)
else:
    df_regression = pd.read_csv(f"../results/{DATA}_error_analysis.csv")

In [5]:
y, X = dmatrices("residuals_best ~ log_gmean_freq  + length + tokenN_in_sent + model + pos + has_punct + has_num" , data=df_regression, return_type='dataframe')
mod = sm.OLS(y, X)
res = mod.fit()
print(res.summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}                & residuals\_best  & \textbf{  R-squared:         } &      0.009    \\
\textbf{Model:}                        &       OLS        & \textbf{  Adj. R-squared:    } &      0.009    \\
\textbf{Method:}                       &  Least Squares   & \textbf{  F-statistic:       } &      113.5    \\
\textbf{Date:}                         & Tue, 07 Jan 2025 & \textbf{  Prob (F-statistic):} &      0.00     \\
\textbf{Time:}                         &     15:10:10     & \textbf{  Log-Likelihood:    } & -2.4625e+06   \\
\textbf{No. Observations:}             &      672896      & \textbf{  AIC:               } &  4.925e+06    \\
\textbf{Df Residuals:}                 &      672839      & \textbf{  BIC:               } &  4.926e+06    \\
\textbf{Df Model:}                     &          56      & \textbf{                     } &               \\
\textbf{Covariance Type:}              &    nonrobust     & \textbf{      

In [10]:
df_regression[["log_gmean_freq", "length"]].corr()

,log_gmean_freq,length
log_gmean_freq,1.000000,-0.743818
length,-0.743818,1.000000
